In [17]:
import os
import torch
import torchvision

In [18]:
# For simple regression problem
TRAINING_POINTS = 1000

In [19]:
# With this block, we don't need to set device=DEVICE for every tensor.
# But you will still need to avoid accidentally getting int types instead of floating-point types.
torch.set_default_dtype(torch.float32)
if torch.cuda.is_available():
     torch.set_default_device(0)
     print("Running on the GPU")
else:
     print("Running on the CPU")

Running on the CPU


In [20]:
def create_linear_training_data():
    """
    This method simply rotates points in a 2D space.
    Be sure to use MSE in the place of the final softmax layer before testing on this
    data!
    :return: (x,y) the dataset. x is a torch tensor where columns are training samples and
             y is a torch tensor where each column is an output sample, rotated 90 degrees 
             in 2d space from the input point. 
    """
    x = torch.randn((2, TRAINING_POINTS))
    x1 = x[0:1, :].clone()
    x2 = x[1:2, :]
    y = torch.cat((-x2, x1), axis=0)
    return x, y

In [21]:
def create_folded_training_data():
    """
    This method introduces a single non-linear fold into the sort of data created by create_linear_training_data.
    Be sure to use MSE in the place of the final softmax layer before testing on this
    data!
    :return: (x,y) the dataset. x is a torch tensor where columns are training samples and
             y is a torch tensor where columns output samples. 
    """
    x = torch.randn((2, TRAINING_POINTS))
    x1 = x[0:1, :].clone()
    x2 = x[1:2, :]
    x2 *= 2 * ((x2 > 0).float() - 0.5)
    y = torch.cat((-x2, x1), axis=0)
    return x, y

In [22]:
def create_square():
    """
    This is a square example in which the challenge is to determine
    if the points are inside or outside of a square in 2d space.
    insideness is true if the points are inside the square.
    :return: (points, insideness) the dataset. points is a 2xN array of points and insideness is true if the point is inside the square.
    """
    win_x = [2,2,3,3]
    win_y = [1,2,2,1]
    win = torch.tensor([win_x,win_y],dtype=torch.float32)
    win_rot = torch.cat((win[:,1:],win[:,0:1]),axis=1)
    t = win_rot - win # edges tangent along side of poly
    rotation = torch.tensor([[0, 1],[-1,0]],dtype=torch.float32)
    normal = rotation @ t # normal vectors to each side of poly
        # torch.matmul(rotation,t) # Same thing

    points = torch.rand((2,2000),dtype = torch.float32)
    points = 4*points

    vectors = points[:,np.newaxis,:] - win[:,:,np.newaxis] # reshape to fill origin
    insideness = (normal[:,:,np.newaxis] * vectors).sum(axis=0)
    insideness = insideness.T
    insideness = insideness > 0
    insideness = insideness.all(axis=1)
    return points, insideness

In [25]:
def relu(x):
    return torch.maximum(x, torch.tensor(0.0))

In [23]:
if __name__ == '__main__':
    # For this week's lab, you can likely keep ALL your code
    # right here, with all your variables in the global scope 
    # for simplified debugging.
    #
    # The code in this section should NOT be in a helper method.
    # Similarly, the output of each sub-layer and activation function should
    # be in this scope.

    # TODO: You may wish to make each TODO below its own pynb cell.
    # TODO: Build your network.

    # TODO: Select your datasource.
    x_train, y_train = create_linear_training_data()
    print(x_train)
    print(y_train)
    # TODO: Train your network.

    # TODO: Sanity-check the output of your network.
    # You can optionally compute the error on this test data:
    x_test, y_test = create_linear_training_data()

    # But you must computed W*M as discussed in the lab assignment.

    pass # You may wish to keep this line as a point to place a debugging breakpoint.

tensor([[-0.4685, -0.1644,  0.3091,  ...,  0.0128,  0.0573, -1.5272],
        [ 0.9017, -1.6444, -0.7179,  ...,  0.0781,  0.8734,  0.6782]])
tensor([[-0.9017,  1.6444,  0.7179,  ..., -0.0781, -0.8734, -0.6782],
        [-0.4685, -0.1644,  0.3091,  ...,  0.0128,  0.0573, -1.5272]])


### Test 1
Output should be [0.2185, 0.027], J should be 0.18101

In [53]:
X = torch.tensor([[0.105, 0.815]])  
W1 = torch.tensor([[0.75, -0.55, 0.25], [0.25, -0.75, -0.15]], requires_grad=True)
b1 = torch.tensor([-0.1, 0.2, -0.3], requires_grad=True)  
W2 = torch.tensor([[0.1, -0.4], [-0.2, 0.5], [0.3, -0.6]], requires_grad=True)  
b2 = torch.tensor([0.2, 0.1], requires_grad=True)  
Y_true = torch.tensor([[0.815, 0.105]])
lambda_l2 = 0.001


In [57]:
Z1 = X @ W1 + b1
A1 = relu(Z1) 
Z2 = A1 @ W2 + b2
Y_pred = Z2  
mse_loss = torch.mean((Y_pred - Y_true) ** 2)
l2_reg = lambda_l2/2 * (torch.sum(W1)**2 + torch.sum(W2)**2)
total_loss = mse_loss + l2_reg

print("Predicted Output (Y_pred):", Y_pred)
print("True Output (Y_true):", Y_true)
print("MSE Loss:", mse_loss.item())
print("L2 Regularization Loss:", l2_reg.item())
print("Total Loss (MSE + L2 Regularization):", total_loss.item())

Predicted Output (Y_pred): tensor([[0.2183, 0.0270]], grad_fn=<AddBackward0>)
True Output (Y_true): tensor([[0.8150, 0.1050]])
MSE Loss: 0.18109728395938873
L2 Regularization Loss: 6.500001472886652e-05
Total Loss (MSE + L2 Regularization): 0.1811622828245163


These values are very close to the output and loss I calculated by hand. The small differences are due to the rounding I did by hand.

## Test 2

Output should be [-2.0, 0.5]. J should be 2.215.

In [63]:
X = torch.tensor([[1.0, 0.0]])  
W1 = torch.tensor([[1.0, 0.0, 1.0], [0.0, 1.0, 0.0]], requires_grad=True)
b1 = torch.tensor([.5, .5, .5], requires_grad=True)  
W2 = torch.tensor([[-1.0, 0.0], [0.0, -1.0], [-1.0, 0.0]], requires_grad=True)  
b2 = torch.tensor([1.0, 1.0], requires_grad=True)  
Y_true = torch.tensor([[0.0, 1.0]])
lambda_l2 = 0.01

In [64]:
Z1 = X @ W1 + b1
A1 = relu(Z1) 
Z2 = A1 @ W2 + b2
Y_pred = Z2  
mse_loss = torch.mean((Y_pred - Y_true) ** 2)
l2_reg = lambda_l2/2 * (torch.sum(W1)**2 + torch.sum(W2)**2)
total_loss = mse_loss + l2_reg

print("Predicted Output (Y_pred):", Y_pred)
print("True Output (Y_true):", Y_true)
print("MSE Loss:", mse_loss.item())
print("L2 Regularization Loss:", l2_reg.item())
print("Total Loss (MSE + L2 Regularization):", total_loss.item())

Predicted Output (Y_pred): tensor([[-2.0000,  0.5000]], grad_fn=<AddBackward0>)
True Output (Y_true): tensor([[0., 1.]])
MSE Loss: 2.125
L2 Regularization Loss: 0.08999999612569809
Total Loss (MSE + L2 Regularization): 2.2149999141693115


These values are very close to the output and loss I calculated by hand. The small differences are due to the rounding I did by hand.